# Homework Solution 9

Ming Hong (mh4286@nyu.edu)

In [1]:
%pylab inline
import pandas as pd
import numpy as np
import fmt
from sobol_lib import i4_sobol_generate as sobol
from scipy.stats import norm

Populating the interactive namespace from numpy and matplotlib


This homework is to price [synthetic CDO](https://en.wikipedia.org/wiki/Synthetic_CDO) using the one factor Gaussian Copula model. 

A synthetic CDO consists of $n$ CDS, the total loss of the portfolio is defned as:

$$ l(t) = \sum_i^n w_i \tilde {\mathbb{1}}_i(t) (1-r_i(t)) $$

where $w_i$ and $r_i(t)$ are the notional weights and recovery rate of the i-th name in the portfolio. The notional weighs sum up to 1: $\sum_i w_i = 1 $. The $ \tilde {\mathbb{1}}_i(t) $ is the default indicator of the i-th name defaulted *before* time $t$, the default probability is therefore $p_i(t) = \mathbb E[\tilde {\mathbb{1}}_i(t) ]$

For the purpose of this homework, we consider a simplified synthetic CDO that has no coupon payments, therefore the PV of a \$1 notional synthetic CDO tranche with maturity $t$, attachment $a$ and detachment $d$ is:

$$ v(a, d) = \frac{d(t)}{d-a} \min\left((l(t) - a)^+, d-a\right) $$

where $d(t)$ is the discount factor.

The following are the parameters to the synthetic CDO, and a straight forward Monte Carlo pricer:

In [2]:
n = 125
t = 5.
defProbs = 1 - exp(-(np.random.uniform(size=n)*.03)*t)
recovery = 0.4*np.ones(n)
w = 1./n*np.ones(n)
rho = 0.5
discf = .9
npath = 1000

# a list of attachements and detachements, they pair up by elements
attachements = np.array([0, .03, .07, .1, .15, .3])
detachements = np.array([.03, .07, .1, .15, .3, .6])

#portfolio expected loss
el = np.sum(w*defProbs*(1-recovery))
print "portfolio expected loss is ", el

portfolio expected loss is  0.0442351880578


In [3]:
class CDO(object) :
    def __init__(self, w, defProbs, recovery, a, d) :
        self.w = w/np.sum(w)
        self.p = defProbs
        self.rec = recovery
        self.rho = rho
        self.a = a
        self.d = d

    def drawDefaultIndicator(self, z, rho) :
        '''return a list of default indicators given common factor z, using one factor Gaussian Copula
        '''
        e = np.random.normal(size=np.shape(self.p))
        x = z*np.sqrt(self.rho) + np.sqrt(1-self.rho)*e
        return np.less(norm.cdf(x), self.p)

    def portfolioLoss(self, defIndicator) :
        '''compute portfolio loss given default indicators'''
        return np.sum(defIndicator*self.w*(1-self.rec))

    def tranchePV(self, portfLoss, discf) :
        '''compute tranche PV from portfolio loss
        Args:
            portfLoss: the total portfolio loss
            discf: discount factor
        Returns:
            tranche PVs'''
        
        sz = self.d - self.a
        return discf/sz*np.minimum(np.maximum(portfLoss - self.a, 0), sz)

    def drawPV(self, z, rho, discf) :
        ''' compute PV and portfolio Loss conditioned on a common factor z'''
        di = self.drawDefaultIndicator(z, rho)
        pfLoss = self.portfolioLoss(di)
        return self.tranchePV(pfLoss, discf), pfLoss
    
    
cdo = CDO(w, defProbs, recovery, attachements, detachements)

In [4]:
## price the tranches using simulation
def simCDO(cdo, rho, disc, paths) :
    zs = np.random.normal(size=[paths])
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV, _ = cdo.drawPV(z, rho, discf)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths), zs

In [5]:
pv_0, err_0, zs = simCDO(cdo, rho, discf, npath)
df = pd.DataFrame(np.array([cdo.a, cdo.d, pv_0, err_0]), index=['Attach', 'Detach', 'PV', 'MC err'])

fmt.displayDF(df, fmt='4g')

,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4828,0.2573,0.161,0.103,0.03952,0.004241
MC err,0.01228,0.01196,0.0105,0.008492,0.005052,0.001308


## Problem 1

Modify the simCDO function to implement the following variance reduction techniques, and show whether the technique is effective:

For this homework, we only apply the variance reduction in the common market factor $z$, you should not change the random number $e$ that were drawn within the drawDefaultIndicator function, i.e., only modify the simCDO code, re-use but do not modify the CDO class. Unless explicitly mentioned, keep the simulation paths the same as the base case above.

1. anti-thetic variate, reduce the number of paths by half to account for the 2x increase in computation
1. importance sampling, shift $z$ by -1
1. sobol sequence
1. stratified sampling: sample $z$ using an equal sized grid

Compute the **variance** reduction factor for each technique, and comment on the effectiveness of these variance reduction techniques.

### Solution

Among the four variance reduction methods, batching is used in importance sampling, Sobol sequence, and stratified sampling. The simulation paths are only kept the same as the base case in the antithetic scheme as it does not require batching.

**1)** Antithetic variate

In [6]:
def simCDO_antithetic(cdo, rho, disc, paths, zs):
    zs = zs[0:paths]
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV1, _ = cdo.drawPV(z, rho, discf)
        thisPV2, _ = cdo.drawPV(-z, rho, discf)
        thisPV = (thisPV1 + thisPV2)/2
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths)


pv_1, err_1 = simCDO_antithetic(cdo, rho, discf, npath/2, zs)
vrf_1 = err_0**2/(err_1**2) # variance reduction factor
df1 = pd.DataFrame(np.array([cdo.a, cdo.d, pv_1, err_1, vrf_1]), index=['Attach', 'Detach', 'PV', 'MC err', 'VRF'])
fmt.displayDFs(df1, headers=['Antithetic'], fmt='4g')

In the antithetic scheme, the first half of the 1,000 simulation paths from the regular MC are used along with their opposite values to get 500 averaged paths. As can be seen, the MC errors in all the tranches are reduced compared to the values in the base case. The variance reduction factor is the highest (8.323) at the lowest tranche (0-3%), and suddenly decreases to a relatively low level (around 1-2) when moving to the more senior tranches.

**2)** Importance Sampling

In [11]:
def simCDO_IS(cdo, rho, disc, paths, u, b):
    means = np.zeros([b, np.shape(cdo.a)[0]])
    for i in range(b):
        zs_q = np.random.normal(size=paths)
        zs_p = zs_q + u # P sample
        m = np.exp(-u*zs_p + 0.5*u*u) # R-N derivative
        qs = 1./paths*np.ones(paths) # Q weights
        ps = m*qs # P weights
        ps = ps/np.sum(ps) # normalization
            
        pv = np.zeros(np.shape(cdo.a))

        for z,p in zip(zs_p,ps):
            thisPV, _ = cdo.drawPV(z, rho, discf)
            pv += thisPV*p
        means[i,:] = pv
            
    return np.mean(means,0), np.std(means,0)

b = 30 # number of batches
pv_2, err_2 = simCDO_IS(cdo, rho, discf, npath, -1, b)
vrf_2 = err_0**2/(err_2**2) # err_2 is the std of the SAMPLE MEAN
df2 = pd.DataFrame(np.array([cdo.a, cdo.d, pv_2, err_2, vrf_2]), index=['Attach', 'Detach', 'PV', 'MC err', 'VRF'])
fmt.displayDFs(df2, headers=['Importance Sampling'], fmt='4g')

In importance sampling, 30 batches of simulations, each with 1,000 paths, are performed to obtain the sample means and sample standard deviations for the PV of each tranche. As can be seen, the MC errors in the lowest tranche actually become larger compared to the base case. The variance reduction factor is the highest (17.97) at the highest tranche (30%-60%), trending upward when moving from low to high. This makes sense as importance sampling is most effective for rare events, which in this case is the loss suffered by the most senior tranche at 30%-60%.

**3)** Sobol Sequence

In [8]:
def simCDO_Sobol(cdo, rho, disc, paths, b):
    means = np.zeros([b, np.shape(cdo.a)[0]])
    ss = sobol(1,paths*b,0)
    for i in range(b):
        zs = norm.ppf(ss[0,i*paths:(i+1)*paths]).T
        pv = np.zeros(np.shape(cdo.a)) # np.shape(cdo.a): number of tranches
        for z in zs:
            thisPV, _ = cdo.drawPV(z, rho, discf)
            pv += thisPV
        means[i,:] = pv/paths

    return np.mean(means,0), np.std(means,0)

pv_3, err_3 = simCDO_Sobol(cdo, rho, discf, npath, b)
vrf_3 = err_0**2/(err_3**2) # variance reduction factor
df3 = pd.DataFrame(np.array([cdo.a, cdo.d, pv_3, err_3, vrf_3]), index=['Attach', 'Detach', 'PV', 'MC err', 'VRF'])
fmt.displayDFs(df3, headers=['Sobol Sequence'], fmt='4g')

Using the Sobol sequence, 30 batches of simulations, each with 1,000 paths, are performed to obtain the sample means and sample standard deviations for the PV of each tranche. As can be seen, the MC errors are all significantly reduced compared to the base case values. The variance reduction factor is the highest (15.04) at the second highest tranche (15%-30%). Compared to the previous two methods, the effectiveness in variance reduction using Sobol sequence is more consistent across the tranches.

**4)** Stratified Sampling

In [9]:
def stratify(u, bs, shuffle) :
    b = len(bs)
    r = len(u)/b + 1
    sb = []
    
    for i in range(r) :
        if shuffle :
            np.random.shuffle(bs)
        sb = sb + bs.tolist()
            
    return [1.*(i + x)/b for x, i in zip(u, sb)]

def simCDO_SS(cdo, rho, disc, paths, nbins, b):
    means = np.zeros([b, np.shape(cdo.a)[0]])
    for i in range(b):
        u = np.random.uniform(size=paths)
        v = stratify(u, np.arange(nbins), False)
        zs = norm.ppf(v)
        pv = np.zeros(np.shape(cdo.a)) # np.shape(cdo.a): number of tranches
        for z in zs:
            thisPV, _ = cdo.drawPV(z, rho, discf)
            pv += thisPV
            
        means[i,:] = pv/paths
    return np.mean(means,0), np.std(means,0)

nbins = 500
pv_4, err_4 = simCDO_SS(cdo, rho, discf, npath, nbins, b)
vrf_4 = err_0**2/(err_4**2) # variance reduction factor
df4 = pd.DataFrame(np.array([cdo.a, cdo.d, pv_4, err_4, vrf_4]), index=['Attach', 'Detach', 'PV', 'MC err', 'VRF'])
fmt.displayDFs(df4, headers=['Stratified Sampling'], fmt='4g')

Using stratified sampling with 500 buckets (to convert the uniform random number), 30 batches of simulations, each with 1,000 paths, are performed to obtain the sample means and sample standard deviations for the PV of each tranche. As can be seen, the MC errors are all significantly reduced compared to the base case values. The variance reduction factor is the highest (23.93) at the highest tranche (30%-60%), while for most other tranches it also beats the value using the Sobol sequence method. Among all the methods, stratified sampling demonstrates the greatest effectiveness in variance reduction.

## (Extra Credit) Problem 2

Consider a control variate for the problem above. The large pool model assumes that the portfolio is a large homogeneous pool, using the average default rate: $\bar p = \frac{1}{n}\sum_i p_i$. Then the portfolio loss conditioned on market factor $z$ under the large pool model is a determinsitic scalar:

$$ l(z) = (1-r)\Phi\left(\frac{\Phi^{-1}(\bar p) - \sqrt \rho z}{\sqrt{1-\rho}}\right)$$

where $r$ is the constant recovery of all names. $\Phi()$ is the normal CDF function; $\Phi^{-1}()$ is its inverse. The tranche PVs can then be computed from the $l(z)$.

Please investigate if the large pool model can be used as an effective control variate. Does it work better for some tranches?

Hint: to answer this question, you only need to compute the correlation between the actual and control variates. 